<a href="https://colab.research.google.com/github/cyclingdata/kaggle_fraud_modelling/blob/master/Calculate_submission_catboost_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install catboost shap ipywidgets sklearn
!jupyter nbextension enable --py widgetsnbextension

     |████████████████████████████████| 62.5MB 627kB/s 
     |████████████████████████████████| 245kB 38.8MB/s 
  Created wheel for shap: filename=shap-0.30.1-cp36-cp36m-linux_x86_64.whl size=360453 sha256=feb500d8b97e9d1ec930116314e38750ebc037154bed3203f5185328b8baea27
  Stored in directory: /root/.cache/pip/wheels/62/94/5e/feb9af12d63a719a32266a29ca564b4dc37b4755052aca6859
Successfully built shap
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [0]:
# Authenticate.
from google.colab import auth
auth.authenticate_user()

In [0]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import pickle
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import pickle
from sklearn.ensemble import RandomForestClassifier

Read data

In [0]:
gcs_data = "gs://data-fraud-detection/test_transaction.csv"
!gsutil cp $gcs_data test.csv
test_transaction=pd.read_csv('test.csv', low_memory=False)
!rm test.csv
gcs_data = "gs://data-fraud-detection/test_identity.csv"
!gsutil cp $gcs_data test.csv
test_identity=pd.read_csv('test.csv', low_memory=False)
!rm test.csv

Copying gs://data-fraud-detection/test_transaction.csv...
/ [1 files][584.8 MiB/584.8 MiB]   23.4 MiB/s                                   
Operation completed over 1 objects/584.8 MiB.                                    
Copying gs://data-fraud-detection/test_identity.csv...
\ [1 files][ 24.6 MiB/ 24.6 MiB]                                                
Operation completed over 1 objects/24.6 MiB.                                     


In [0]:
!gsutil cp gs://data-fraud-detection/to_impute.pickle to_impute.pickle
to_impute = pickle.load(open('to_impute.pickle','rb'))
!rm to_impute.pickle
!gsutil cp gs://data-fraud-detection/dict_freq_ca.pickle dict_freq_cat.pickle
dict_freq_cat = pickle.load(open('dict_freq_cat.pickle','rb'))
!rm dict_freq_cat.pickle

Copying gs://data-fraud-detection/to_impute.pickle...
/ [1 files][ 16.1 KiB/ 16.1 KiB]                                                
Operation completed over 1 objects/16.1 KiB.                                     
Copying gs://data-fraud-detection/dict_freq_ca.pickle...
- [1 files][  2.9 MiB/  2.9 MiB]                                                
Operation completed over 1 objects/2.9 MiB.                                      


In [0]:
!gsutil cp gs://data-fraud-detection/numerical_features.pickle numerical_features.pickle
!gsutil cp gs://data-fraud-detection/categorical_features.pickle categorical_features.pickle 
numerical_features = pickle.load(open('numerical_features.pickle','rb'))
categorical_features = pickle.load(open('categorical_features.pickle','rb'))
!gsutil cp gs://data-fraud-detection/dict_freq_ca.pickle dict_freq_cat.pickle
dict_freq_cat = pickle.load(open('dict_freq_cat.pickle','rb'))

Copying gs://data-fraud-detection/numerical_features.pickle...
/ [1 files][  5.8 KiB/  5.8 KiB]                                                
Operation completed over 1 objects/5.8 KiB.                                      
Copying gs://data-fraud-detection/categorical_features.pickle...
- [1 files][  2.2 MiB/  2.2 MiB]                                                
Operation completed over 1 objects/2.2 MiB.                                      
Copying gs://data-fraud-detection/dict_freq_ca.pickle...
- [1 files][  2.9 MiB/  2.9 MiB]                                                
Operation completed over 1 objects/2.9 MiB.                                      


In [0]:
test_identity["device_info_cut"]=test_identity.DeviceInfo.str[:2]
test_identity['present_identity'] = 1
test_transaction = pd.merge(test_transaction,test_identity,on="TransactionID",how='left')
import datetime
test_transaction['Transaction_hour'] = np.floor(test_transaction['TransactionDT'] / 3600) % 24
START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
test_transaction['TransactionDT'] = test_transaction['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
test_transaction['TransactionDT_Days'] = test_transaction['TransactionDT'].dt.day
import re
test_transaction['D9'] = test_transaction['TransactionDT'].dt.hour
test_transaction['dow'] = test_transaction['TransactionDT'].dt.dayofweek
test_transaction['id_30_alpha'] = test_transaction['id_30'].str.replace('[^a-zA-Z]', '')
test_transaction['id_31_alpha'] = test_transaction['id_31'].str.replace('[^a-zA-Z]', '')
test_transaction['group_os'] = test_transaction["id_30"].str.split(' ', expand = True)[[0]]
test_transaction['group_browser'] = test_transaction["id_31"].str.split(' ', expand = True)[[0]]

test_transaction['uid'] = test_transaction['card1'].astype(str)+'_'+test_transaction['card2'].astype(str)
test_transaction['uid2'] = test_transaction['uid'].astype(str)+'_'+test_transaction['card3'].astype(str)+'_'+test_transaction['card5'].astype(str)
test_transaction['uid3'] = test_transaction['uid2'].astype(str)+'_'+test_transaction['addr1'].astype(str)+'_'+test_transaction['addr2'].astype(str)


test_transaction['present_identity'].fillna(0,inplace=True)
test_transaction['device_name'] = test_transaction['DeviceInfo'].str.split('/', expand=True)[0]
test_transaction['device_version'] = test_transaction['DeviceInfo'].str.split('/', expand=True)[1]

test_transaction.loc[test_transaction['device_name'].str.contains('SM', na=False), 'device_name'] = 'Samsung' 
test_transaction.loc[test_transaction['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung' 
test_transaction.loc[test_transaction['device_name'].str.contains('GT-', na=False), 'device_name'] = 'Samsung' 
test_transaction.loc[test_transaction['device_name'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola' 
test_transaction.loc[test_transaction['device_name'].str.contains('Moto', na=False), 'device_name'] = 'Motorola' 
test_transaction.loc[test_transaction['device_name'].str.contains('moto', na=False), 'device_name'] = 'Motorola' 
test_transaction.loc[test_transaction['device_name'].str.contains('LG-', na=False), 'device_name'] = 'LG' 
test_transaction.loc[test_transaction['device_name'].str.contains('rv:', na=False), 'device_name'] = 'RV' 
test_transaction.loc[test_transaction['device_name'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei' 
test_transaction.loc[test_transaction['device_name'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei' 
test_transaction.loc[test_transaction['device_name'].str.contains('-L', na=False), 'device_name'] = 'Huawei' 
test_transaction.loc[test_transaction['device_name'].str.contains('Blade', na=False), 'device_name'] = 'ZTE' 
test_transaction.loc[test_transaction['device_name'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE' 
test_transaction.loc[test_transaction['device_name'].str.contains('Linux', na=False), 'device_name'] = 'Linux' 
test_transaction.loc[test_transaction['device_name'].str.contains('XT', na=False), 'device_name'] = 'Sony' 
test_transaction.loc[test_transaction['device_name'].str.contains('HTC', na=False), 'device_name'] = 'HTC' 
test_transaction.loc[test_transaction['device_name'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'
test_transaction.loc[test_transaction.device_name.isin(test_transaction.device_name.value_counts()[test_transaction.device_name.value_counts() < 200].index), 'device_name'] = "Others"


test_transaction['Dollars'] = test_transaction['TransactionAmt'].apply(np.floor).astype('int')
test_transaction['Cents'] = (test_transaction['TransactionAmt'].round(2)*100- 100*test_transaction['Dollars']).astype('int')
test_transaction['Nb_decimals_cents'] = (test_transaction['TransactionAmt']-test_transaction['Dollars']).astype('str').str.len()
test_transaction['Dollars_last_digit'] = test_transaction['Dollars'].mod(10)  
test_transaction['Dollars_first_digit'] = test_transaction['Dollars'].astype('str').str[0] 
test_transaction[['Dollars','Cents','TransactionAmt','Nb_decimals_cents','Dollars_last_digit','Dollars_first_digit']].head(10)


test_transaction['screen_width'] = (test_transaction['id_33'].str.split('x', expand=True)[0]).astype(float)
test_transaction['screen_height'] = (test_transaction['id_33'].str.split('x', expand=True)[1]).astype(float)

In [0]:
test_transaction.drop(columns=['device_version'],inplace=True)

In [0]:

import math
for feature in numerical_features:
  if feature[:5] != 'count':
    value_to_impute = to_impute[feature]
    if math.isnan(value_to_impute):
      numerical_features.remove(feature)
    else:
      test_transaction[feature].fillna(value_to_impute,inplace=True)

#numerical_features = to_impute.keys().to_list()

for feature in categorical_features:
  print(feature)
  test_transaction[feature] = test_transaction[feature].astype("str")
  test_transaction[feature]=test_transaction[feature].str.replace(' ', '_')
  test_transaction[feature]=test_transaction[feature].str.replace('.0', '',regex=False)
  test_transaction[feature]=test_transaction[feature].str.replace('.', '_',regex=False)
  test_transaction[feature]=test_transaction[feature].str.replace(':','_',regex=False)
  test_transaction[feature]=test_transaction[feature].str.replace('/','_',regex=False)

card1
card2
card3
card4
card5
card6
addr1
addr2
P_emaildomain
R_emaildomain
ProductCD
M1
M2
M3
M4
M5
M6
M7
M8
M9
DeviceType
DeviceInfo
device_info_cut
dow
id_30_alpha
id_31_alpha
group_os
group_browser
uid
uid2
uid3
TransactionDT_Days
device_name
Dollars_first_digit
Dollars_last_digit
Cents
Dollars
id_12
id_13
id_14
id_15
id_16
id_17
id_18
id_19
id_20
id_21
id_22
id_23
id_24
id_25
id_26
id_27
id_28
id_29
id_30
id_31
id_32
id_33
id_34
id_35
id_36
id_37
id_38


In [0]:
%%time

min_cat = 0.0005
valid_values = {}
old_numerical_features=numerical_features.copy()
for cat in categorical_features:
    print(cat)
    test_transaction[cat].fillna('missing',inplace=True)
    counts = dict_freq_cat[cat] 
    #rare_values = counts[counts['count_'+cat] <= min_cat][cat].tolist()
    #test_transaction[cat].replace(rare_values, "RARE_VALUE",inplace=True)
    test_transaction[cat] = test_transaction[cat].astype('str')
    test_transaction = pd.merge(test_transaction,counts,on=cat,how='left')
    test_transaction['count_'+cat].fillna(0,inplace=True)

    

card1
card2
card3
card4
card5
card6
addr1
addr2
P_emaildomain
R_emaildomain
ProductCD
M1
M2
M3
M4
M5
M6
M7
M8
M9
DeviceType
DeviceInfo
device_info_cut
dow
id_30_alpha
id_31_alpha
group_os
group_browser
uid
uid2
uid3
TransactionDT_Days
device_name
Dollars_first_digit
Dollars_last_digit
Cents
Dollars
id_12
id_13
id_14
id_15
id_16
id_17
id_18
id_19
id_20
id_21
id_22
id_23
id_24
id_25
id_26
id_27
id_28
id_29
id_30
id_31
id_32
id_33
id_34
id_35
id_36
id_37
id_38
CPU times: user 2min 12s, sys: 2.91 s, total: 2min 15s
Wall time: 2min 15s


In [0]:
import catboost
from catboost import CatBoostClassifier
!gsutil cp  gs://data-fraud-detection/catboost_classifier_lr01.pickle catboost_classifier_lr01.pickle
model = pickle.load(open('catboost_classifier_lr01.pickle','rb'))

Copying gs://data-fraud-detection/catboost_classifier_lr01.pickle...
\
Operation completed over 1 objects/4.8 GiB.                                      


In [0]:
tID = test_transaction.TransactionID

test_transaction.drop(columns=['TransactionID'],inplace=True)

In [0]:
len(test_transaction.columns.tolist())

516

In [0]:
pred_test= model.predict_proba(test_transaction)[:,1].tolist()
submission=pd.DataFrame({"TransactionID": tID,"isFraud":pred_test})
submission.head()

,TransactionID,isFraud
0,3663549,0.001452
1,3663550,0.008484
2,3663551,0.000888
3,3663552,0.001765
4,3663553,0.007048


In [0]:
submission.to_csv("submission.csv",index=False)
!gsutil cp submission.csv gs://data-fraud-detection/submission_catboost.csv

Copying file://submission.csv [Content-Type=text/csv]...
-
Operation completed over 1 objects/14.0 MiB.                                     
